# QA - gluon
* 참조: http://gluon.mxnet.io/chapter08_computer-vision/visual-question-answer.html
* 강의: http://m.edwith.org/deeplearningchoi/lecture/15580

## DPPnet (Attention Based Method)

* 일반적으로 이미지를 인식하기 위해서는 이미 학습되어 있는 기존의 이미지 분류기 (예를 들어, VGG, RESNET 등)의 마지막 layer를 transfer learning하는 방식으로 분류 진행
* 하지만, VQA에서는 Task가 질문에 의존하므로, 마지막 layer를 질문의 정보를 포함하여 학습해보자 하는 것이 아이디어
<img src = '../images/DPP_idea.png' alter = 'DPP idea' width = '500px'> </img>

* 질문에서 feature를 뽑고, 뽑힌 feature를 바탕으로 regression해서 parameter를 찾는 방법
  * 여기에서 regression이란 identity activation을 가지는 hidden layer가 없는 MLP를 의미함
  * 이 Regression의 결과물을 image network의 마지막 layer로 활용
  * Feature를 뽑는 방법은 BOW, CBOW (embedding한 word), CNN, RNN, GRU, LSTM 등을 활용

## 문제점

* Parameter explosion
  * 추정해야 할 parameter의 개수가 엄청나게 증가함
  <img src ='../images/DPP_parameter_explosion.png' alt = 'Parameter explosion' width = '500px' border = '2px'></img>
  
  * Hashing trick을 이용해서 적은 숫자의 parameter로 fully network로 구성하여 parameter가 폭발적으로 증가하는 문제점을 극복
  <img src ='../images/DPP_hashing_trick.png' alt = 'Hashing trick' width = '500px' border = '2px'></img>

## DPP Structure

<img src = '../images/DPP_structure.png' width = '700px' border = '2px'>

# MCB

## Multi modality를 가진 두 data를 결합하는 방법

* 질문과 이미지라는 서로 다른 multi modality를 가지는 것들을 엮는 작업으로 다음과 같은 방법들을 생각해 볼 수 있음
  * 가장 쉽게는 두가지 정보를 하나의 vector로 **concat**하는 방법 $\Rightarrow$ 단어 정보와 image 정보의 상관관계가 무시됨
    <img src = '../images/MCB_concat.png' width = '500px'></img>
    
  * 두가지 정보를 **elementwise multiplication**하는 방법 $\Rightarrow$ 같은 위치의 원소들이 같은 의미를 지니지는 않을 것임
    <img src = '../images/MCB_elementwise_multiplication.png' width = '500px'></img>
    
  * 두가지 정보의 **outer product**를 사용하는 방법 $\Rightarrow$ 모든 정보의 interacion을 고려할 수 있음
    * 하지만 parameter가 너무 커지는 단점
    <img src = '../images/MCB_outer_product.png' width = '500px'></img>
    
> 다음의 sketch count trick으로 parameter의 갯수가 폭증하는 것을 방지함

## Count Sketch

* Outer product의 단점
  - feature의 외적을 보다 작은 size의 행렬로 나타내는 방법이 필요하고 논문에서는 Multimodal Compact Bilinear Pooling을 제시
  - size를 줄이는 방법으로 count sketch라는 방법($\psi$)이 사용
  
> **Counter Sketch**  
<img src = '../images/MCB_count_sketch.png' width = '500px'></img>


  - 결국 하고 싶은 것은, 외적에 count sketch를 적용하고 싶은 것임
  $$\psi(x\otimes y,h,s)$$
  
  - 그런데, 이렇게 하면 2048 $\times$ 2048 차원의 matrix를 계산하고, count sketch 변환을 해야 하는데 그러면 $h$와 $s$도 2048 $\times$ 2048 차원이 되어야 함
  - 보다 효율적인 방법을 위해 다음과 같은 bypass를 고려함
$$\psi(x,h,s)\star \psi(y,h,s)$$

    - 위 식의 의미는 먼저 count sketch 연산을 vector에 대해서 수행을 하고, 
    - 그 다음에 $\star$ 연산을 진행하고자 함
> **$\star$ 연산은 FFT를 이용한 연산으로 다음과 같이 수행할 수 있음** 
> 두 vector를 외적한 후 count sketch 변환을 거치는 것과 sketch count를 적용한 후 convolution하는 것이 동일하다는 것을 보임 (Pham & Pagh, 2013)
$$a \star b = FFT^{-1}(FFT(a) \odot FFT(b)$$
<img src = ' ../images/MCB_using_sketch.png' width = '500px'></img>

  - $\Rightarrow$ Sketch count의 outer product를 구하는 것보다, 각 sketch count의 FFT 변환의 element-wise 곱의 IFFT를 구하면 됨
      * NOTE: FFT
      ![](../images/fft.png 'Fast Fourier Transform')
      
  * Counter sketch
    * 임의로 정한 $s$와 $h$라는 vector에 의해 정해진 변환으로 원래 vector를 일정한 rule로 바꿔주는 역할
      * $s$는 해당 위치의 원소의 부호를 바꾸는 역할 (-1과 1로 이루어짐)
      * $h$는 원소의 순서를 정하는 역할 (1부터 임의로 정한 숫자 d 사이의 숫자)로 sketch count 결과물의 dimension임
      ![](../images/mcb_paper_fig.png 'mcb concept in paper')
    

* Image pretrained model:
  * VGG net
* Question pretrained model:
  * Word2Vec
* Try two models
  * Model 1: Just concat image and question features and use multilayer perceptron to predict the answer
  * Model 2: multimodel compact bilinear pooling(MCB)
    * use count sketch to estimate the outer product of the image and question features 
    ![](../images/mcb.png "MCB")
    * MCB:
    $$\psi(x\otimes y,h,s)=\psi(x,h,s)\star \psi(y,h,s)$$

## MCB with Attention

![](../images/mcb_attention.png 'mcb with attention')